In [18]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records

fix_block_number = None
fix_block_number = 17882772 - 960  # mainnet
fix_block_number = 120210057 - 1960  # arbitrum
fix_block_number = 45902962 - 1030  # polygon

cmf_param = {
    'block_number': fix_block_number,
    # 'chain_id': 1,
    # 'api_url': 'http://localhost:8700',
    # 'chain_id': 42161,
    # 'api_url': 'http://localhost:8700',
    'chain_id': 137,
    'use_local_models': '-',
}

context, _model_loader = create_cmf(cmf_param, True)

fix_block_number = context.block_number

print(fix_block_number)

Credmark context created with 
- chain_id=137
- block_number=45901932
- chain_to_provider_url=https://nd...10ab
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=https://gateway.credmark.com
- use_local_models=-

45901932


## Vault Dashboard


```bash
time curl -X 'POST' https://gateway.credmark.com/v1/model/run \
-H 'accept: application/json' \
-H 'Content-Type: application/json' \
-H "Authorization: Bearer $CREDMARK_API_KEY" \
-d '{ "slug": "ichi.vaults-performance", "chainId": 137, "blockNumber": "latest", "input": {} }' | jq
```


In [19]:
result = context.run_model('ichi.vaults-performance', {})
df_vaults_performance = pd.DataFrame(result['data'])

df_digest = (df_vaults_performance
             .loc[:, ['vault', 'pool', 'tvl', 'token0_symbol', 'token1_symbol',
                      'allowed_token', 'days_since_first_deposit', ] +
                  ['qty_hold', 'qty_vault'] +
                  [f'return_rate{i}' for i in ['']] +
                  [f'irr_cashflow{i}' for i in ['']] +
                  [f'irr_cashflow_non_zero{i}' for i in ['']]]
             .sort_values('tvl', ascending=False)
             .reset_index(drop=True)
             .assign(
                 tvl=lambda df: round(df.tvl, 0),
                 # days_since_first_deposit=lambda df: df.days_since_first_deposit.astype(int),
                 # qty_hold=lambda df: round(df.qty_hold, 1),
                 # qty_vault=lambda df: round(df.qty_vault, 1),
                 # return_rate=lambda df: round(df.return_rate * 100, 1),
                 # irr_cashflow=lambda df: round(df.irr_cashflow * 100, 1),
                 # irr_cashflow_non_zero=lambda df: (df.irr_cashflow_non_zero * 100).astype(float)
             )
             )
df_digest.to_csv(
    f'df_vaults_performance_updated6_{cmf_param["chain_id"]}.csv', index=False)
df_digest


,vault,pool,tvl,token0_symbol,token1_symbol,allowed_token,days_since_first_deposit,qty_hold,qty_vault,return_rate,irr_cashflow,irr_cashflow_non_zero
0,0x9c90975b13d04d7d535359887c236ac51f2298ce,0xa6b96e60648e11055e82f1b7b226a2aa453a29bb,246276.0,wstETH,FBX,wstETH,58.987106,1000,1024.197130,0.024197,0.066956,0.066956
1,0x6bd9239e675092306833c0b29aa552e64d885599,0x16905a1e7f86abc4722f27943d7f42e1368e0090,98344.0,WETH,POP,POP,88.577187,1000,1032.674364,0.032674,0.174232,0.174232
2,0x9ff3c1390300918b40714fd464a39699ddd9fe00,0x86f1d8390222a3691c28938ec7404a1661e618e0,64497.0,WMATIC,WETH,WMATIC,163.012234,1000,1072.302544,0.072303,-0.146090,-0.152871
3,0x74f9d8861d42ac09759ade7809a67cf053dc7ba5,0x0312692e9cadd3ddaace2e112a4e36397bd2f18a,31176.0,SUSHI,WETH,SUSHI,163.010428,1000,1305.632617,0.305633,0.676710,0.675977
4,0x4aef5144131db95c110af41c8ec09f46295a7c4b,0x61f7d1f537e959d62265a76bf1ac40eb3e338de7,22728.0,ICHI,WBTC,WBTC,491.777280,1000,1138.755625,0.138756,0.037143,0.034999
5,0xf461063819ffbc6e22704ade1861b0df3bac9585,0x4fe1269a585b141f11c3e144158f9f8823c7c0e7,18659.0,WETH,BAL,BAL,163.004734,1000,1174.135516,0.174136,0.397994,0.413883
6,0x5394eb43700ce8fbbf4cb83e8b52ea73b71426b6,0x1aae2bedfedb0bb1c07f599445fbb740033035fa,17734.0,ICHI,WBTC,ICHI,164.694201,1000,1027.346521,0.027347,0.051932,0.062699
7,0xf7b1ab2545451b60345fa3ab8c5210d53c703c98,0xfc99d1c02d27de07dfe0dcd878cde86ee59c5f6b,16119.0,CRV,WETH,CRV,143.778681,1000,1109.719094,0.109719,0.207446,0.207446
8,0xa777b0f7f746e93ad30292e9a07b4e336f9511a1,0x215ea98c2a293adee2e33f38c2e18c0564361a2a,15782.0,fxWEN,WETH,fxWEN,83.913530,1000,2617.878797,1.617879,0.954218,0.954218
9,0xac6c0264511eeec305da9afc2e1aba08409f99f6,0x3d86a4b8c1b55509792d57e0c038128cc9c14fe7,13579.0,WMATIC,ICHI,ICHI,164.692951,1000,1025.970308,0.025970,0.027528,0.027528


In [21]:
result = context.run_model('ichi.vaults-performance',
                           {"days_horizon": [7, 30, 60, 90]})
df_vaults_performance = pd.DataFrame(result['data'])

df_digest = (df_vaults_performance
             .loc[:, ['vault', 'tvl', 'token0_symbol', 'token1_symbol',
                      'allowed_token', 'days_since_first_deposit', ] +
                  ['qty_hold', 'qty_vault', 'qty_vault_7', 'qty_vault_30', 'qty_vault_60'] +
                  [f'return_rate{i}' for i in ['', '_7', '_30', '_60']] +
                  [f'irr_cashflow{i}' for i in ['', '_7', '_30', '_60']] +
                  [f'irr_cashflow_non_zero{i}' for i in ['', '_7', '_30', '_60']]]
             .sort_values('tvl', ascending=False)
             .reset_index(drop=True)
             .assign(
                 tvl=lambda df: round(df.tvl, 0),
                 days_since_first_deposit=lambda df: df.days_since_first_deposit.astype(
                     int),
                 qty_hold=lambda df: round(df.qty_hold, 1),

                 qty_vault=lambda df: round(df.qty_vault, 1),
                 qty_vault_7=lambda df: round(df.qty_vault_7, 1),
                 qty_vault_30=lambda df: round(df.qty_vault_30, 1),
                 qty_vault_60=lambda df: round(df.qty_vault_60, 1),

                 return_rate=lambda df: round(df.return_rate * 100, 1),
                 return_rate_7=lambda df: round(df.return_rate_7 * 100, 1),
                 return_rate_30=lambda df: round(df.return_rate_30 * 100, 1),
                 return_rate_60=lambda df: round(df.return_rate_60 * 100, 1),

                 irr_cashflow=lambda df: round(df.irr_cashflow * 100, 1),
                 irr_cashflow_7=lambda df: round(df.irr_cashflow_7 * 100, 1),
                 irr_cashflow_30=lambda df: round(df.irr_cashflow_30 * 100, 1),
                 irr_cashflow_60=lambda df: round(df.irr_cashflow_60 * 100, 1),

                 irr_cashflow_non_zero=lambda df: (
                     df.irr_cashflow_non_zero * 100).astype(float),
                 irr_cashflow_non_zero_7=lambda df: (
                     df.irr_cashflow_non_zero_7 * 100).astype(float),
                 irr_cashflow_non_zero_30=lambda df: (
                     df.irr_cashflow_non_zero_30 * 100).astype(float),
                 irr_cashflow_non_zero_60=lambda df: (
                     df.irr_cashflow_non_zero_60 * 100).astype(float),
             )
             # .to_csv('df_vaults_performance_updated6.csv', index=False)
             )
df_digest


,vault,tvl,token0_symbol,token1_symbol,allowed_token,days_since_first_deposit,qty_hold,qty_vault,qty_vault_7,qty_vault_30,...,return_rate_30,return_rate_60,irr_cashflow,irr_cashflow_7,irr_cashflow_30,irr_cashflow_60,irr_cashflow_non_zero,irr_cashflow_non_zero_7,irr_cashflow_non_zero_30,irr_cashflow_non_zero_60
0,0x9c90975b13d04d7d535359887c236ac51f2298ce,246276.0,wstETH,FBX,wstETH,58,1000,1024.2,1002.6,1011.4,...,1.1,NaN,6.7,14.6,14.0,NaN,6.695550,14.648620,14.007912,NaN
1,0x6bd9239e675092306833c0b29aa552e64d885599,98344.0,WETH,POP,POP,88,1000,1032.7,1003.0,1017.0,...,1.7,1.3,17.4,16.7,22.8,8.4,17.423240,16.694119,22.813438,8.363018
2,0x9ff3c1390300918b40714fd464a39699ddd9fe00,64497.0,WMATIC,WETH,WMATIC,163,1000,1072.3,1003.7,977.5,...,-2.2,-1.3,-14.6,20.6,-77.3,-33.4,-15.287106,15.782386,-84.487887,-45.024956
3,0x74f9d8861d42ac09759ade7809a67cf053dc7ba5,31176.0,SUSHI,WETH,SUSHI,163,1000,1305.6,998.0,1063.5,...,6.3,12.6,67.7,-9.9,112.3,98.5,67.597689,-9.909706,-0.860820,35.787884
4,0x4aef5144131db95c110af41c8ec09f46295a7c4b,22728.0,ICHI,WBTC,WBTC,491,1000,1138.8,1056.1,1058.3,...,5.8,-1.8,3.7,1623.4,99.1,-10.7,3.499861,1623.371725,99.141578,-10.704954
5,0xf461063819ffbc6e22704ade1861b0df3bac9585,18659.0,WETH,BAL,BAL,163,1000,1174.1,1006.9,1029.5,...,2.9,5.8,39.8,41.2,42.0,40.3,41.388261,-99.766016,-68.101893,-32.822228
6,0x5394eb43700ce8fbbf4cb83e8b52ea73b71426b6,17734.0,ICHI,WBTC,ICHI,164,1000,1027.3,1000.0,1000.0,...,0.0,0.0,5.2,-0.0,0.0,-0.0,6.269915,NaN,-99.655977,-94.134776
7,0xf7b1ab2545451b60345fa3ab8c5210d53c703c98,16119.0,CRV,WETH,CRV,143,1000,1109.7,990.8,1008.8,...,0.9,2.2,20.7,-38.0,11.3,12.9,20.744604,-37.983461,11.347357,12.909987
8,0xa777b0f7f746e93ad30292e9a07b4e336f9511a1,15782.0,fxWEN,WETH,fxWEN,83,1000,2617.9,1022.4,1085.8,...,8.6,14.3,95.4,216.9,172.4,125.5,95.421751,216.853860,172.354180,125.530734
9,0xac6c0264511eeec305da9afc2e1aba08409f99f6,13579.0,WMATIC,ICHI,ICHI,164,1000,1026.0,1014.0,1011.2,...,1.1,-0.7,2.8,1.8,-3.0,-12.1,2.752773,1.775055,-2.956943,-12.147987
